# Module 10 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [1]:
from unification import parse, unification, is_variable
from copy import deepcopy

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

In [2]:
def forward_planner( start_state, goal, actions, debug=False):
    return []

You will be solving the problem from above. Here is the start state:

In [3]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [4]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [5]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

### Parsing the input data
First, we need to use the provided functions `parse()` and `unification()` to be able to handle the inputs. 
#### parse_actions(actions)
Takes a dictionary of `actions` and traverses it while using `parse()` to convert the inputs into s expressions that can be used later for further evaluation.

In [6]:
def parse_actions(actions):
    for action in actions.keys():   
        for key in actions[action].keys():
            if isinstance(actions[action][key],list):
                actions[action][key] = [parse(s) for s in actions[action][key]]
            else:
                actions[action][key] = parse(actions[action][key])
    return actions

In [7]:
sample_actions = {'fly':{'action':"(fly ?plane ?from ?to)",
                       'conditions':['(plane ?plane)','(airport ?to)','(airport ?from)','(at ?plane ?from)'],
                       'add':['(at ?plane ?to)'],
                       'delete':['(at ?plane ?from)']}}

sample_actions = parse_actions(sample_actions)

assert sample_actions == {'fly': {'action': ['fly', '?plane', '?from', '?to'],
                                                 'conditions': [['plane', '?plane'],
                                                                ['airport', '?to'],
                                                                ['airport', '?from'],
                                                                ['at', '?plane', '?from']],
                                                 'add': [['at', '?plane', '?to']],
                                                 'delete': [['at', '?plane', '?from']]}}

### Generating valid actions
As indicated above in the instructions, we need to implement a two-fold state space search, the functions in this section will provide a mechanism to search through actions with validation, and generating a set of possible actions to take. In order to store the information. The overall function that implements this is `search_actions()`. Each possible action will be store in an `action_node` while each particular node will be stored in a `node`.

#### build_action_node(action, actions)
An `action_node` corresponds to a dictionary with the unfication provided substitutions stored in the `substitutions` key, and the conditions of the given `action` stored in the `conditions` key. At the point of creation, a node has no substitutions and those will be filled in once the action node is unified later in the procedure. `action` corresponds to the name of the action to be in the node and `actions` corresponds to the dictionary of all available actions.

In [8]:
def build_action_node(action, actions):
    action_node = {'substitutions':{},'conditions':actions[action]['conditions']}
    return action_node

In [9]:
sample_action_node = build_action_node('fly',sample_actions)
assert sample_action_node == {'substitutions': {},
                              'conditions': [['plane', '?plane'],
                                             ['airport', '?to'],
                                             ['airport', '?from'],
                                             ['at', '?plane', '?from']]}

In [10]:
def add_substitutions(dict1, dict2):
    for key in dict1:
        if key in dict2 and dict1[key]!=dict2[key]:
            return None
        else:
            dict1.update(dict2)
            return dict1

In [11]:
dict1 = {'?y': 'Barney', '?x': 'Bam_Bam'}
dict2 = {'?z': 'William'}
assert add_substitutions(dict1, dict2) == {'?y': 'Barney', '?x': 'Bam_Bam', '?z': 'William'}

dict1 = {'?y': 'Barney', '?x': 'Bam_Bam'}
dict2 = {'?y': 'William'}
assert add_substitutions(dict1, dict2) == None

dict1 = {'?y': 'Barney', '?x': 'Bam_Bam'}
dict2 = {'?y': 'Barney','?z':'Rigus'}
assert add_substitutions(dict1, dict2) == {'?y': 'Barney', '?x': 'Bam_Bam', '?z': 'Rigus'}

#### `find_action_successors(action_node, predicates)`
`find_action_successors()` looks and creates all the possible combination successors of a given `action_node` for given `predicates` on a state space. The function makes use of the `unification()` function to make replacements as needed when generating action successors. Keep in mind that this function does not take into account validation of actions. This will be done later on in the program. The function returns a dictionary of substitutions in which the dictionary of substitutions is not None.

In [12]:
def find_action_successors(action_node, predicates):
    successors = []
    if len(action_node['conditions']) == 0:
        return successors
    condition = action_node['conditions'][0]
    for predicate in predicates:        
        unified = unification(condition, predicate)
        if not unified:
            continue
        new_substitutions = add_substitutions(unified, action_node['substitutions'])
        if new_substitutions is not None:
            successor = {'substitutions':unified, 'conditions':action_node['conditions'][1:]}
            successors.append(successor)
    return successors

In [13]:
sample_state = ['(plane 1973)','(plane 2749)',
                '(airport SFO)','(airport JFK)','(airport ORD)',
                '(at 1973 SFO)','(at 2749 JFK)','(at 97 ORD)','(at 1211 SFO)']
sample_state = [parse(predicate) for predicate in sample_state]

sample_action_successors = find_action_successors(sample_action_node, sample_state)

assert sample_action_successors == [{'substitutions': {'?plane': '1973'},
                                     'conditions': [['airport', '?to'],
                                                    ['airport', '?from'],
                                                    ['at', '?plane', '?from']]},
                                    {'substitutions': {'?plane': '2749'},
                                     'conditions': [['airport', '?to'],
                                                    ['airport', '?from'],
                                                    ['at', '?plane', '?from']]}]

#### `search_actions(state, action, actions)`
The `search_actions()` function serves the purpose of creating possible combinations of actions and agents given the `state`, `action`, and available `actions`. The function makes use of the previously defined `find_action_successors()` function to find all available combinations. These combinations are later evaluated in `build_state()` below. This function therefore implements the inner state search as it initiates a frontier, and then uses DFS to navigate the frontier until all available actions are exhausted. The function returns a list of viable substitution lists. 

In [14]:
def search_actions(state, action, actions):
    viable = []
    frontier = [build_action_node(action, actions)]
    while len(frontier) != 0:
        current_node = frontier.pop(0)
        if len(current_node['conditions']) == 0:
            viable.append(current_node['substitutions'])
        for action_node in find_action_successors(current_node, state):
            frontier.append(action_node)
    return viable       

In [15]:
sample_search_actions = search_actions(sample_state, 'fly', sample_actions)
# there are a total of 2 planes 
# a total of 3 airports to be from
# and a total of 3 airports to be to
# the total number of available actions (although not logically sound)
# is 2*3*3 = 18
assert sample_search_actions == [{'?plane': '1973', '?from': 'SFO', '?to': 'SFO'},
                                 {'?plane': '1973', '?from': 'JFK', '?to': 'SFO'},
                                 {'?plane': '1973', '?from': 'ORD', '?to': 'SFO'},
                                 {'?plane': '1973', '?from': 'SFO', '?to': 'JFK'},
                                 {'?plane': '1973', '?from': 'JFK', '?to': 'JFK'},
                                 {'?plane': '1973', '?from': 'ORD', '?to': 'JFK'},
                                 {'?plane': '1973', '?from': 'SFO', '?to': 'ORD'},
                                 {'?plane': '1973', '?from': 'JFK', '?to': 'ORD'},
                                 {'?plane': '1973', '?from': 'ORD', '?to': 'ORD'},
                                 {'?plane': '2749', '?from': 'SFO', '?to': 'SFO'},
                                 {'?plane': '2749', '?from': 'JFK', '?to': 'SFO'},
                                 {'?plane': '2749', '?from': 'ORD', '?to': 'SFO'},
                                 {'?plane': '2749', '?from': 'SFO', '?to': 'JFK'},
                                 {'?plane': '2749', '?from': 'JFK', '?to': 'JFK'},
                                 {'?plane': '2749', '?from': 'ORD', '?to': 'JFK'}, 
                                 {'?plane': '2749', '?from': 'SFO', '?to': 'ORD'}, 
                                 {'?plane': '2749', '?from': 'JFK', '?to': 'ORD'},
                                 {'?plane': '2749', '?from': 'ORD', '?to': 'ORD'}]
assert len(sample_search_actions) == 18

### Generating Valid States
Up to now we have generated a series of functions which do a state space search for possible actions to take. This corresponds to the inner layer state space search in the problem definition above. Now, we move onto the outer state space search as we have the ingredients to do so. The following functions generate states, find successors, and finally implement the entire forward plan.

#### `build_state(state, action, substitutions, actions, valid=True)`
This is quite an important function, as it implements the action validation. `build_state()` takes in a current `state`, an `action` to be taken, a series of `substitutions` as created by `search_actions()`, the universe of `actions` and generates a new valid state whenever `valid` is true (which it is by default).
The function has the three following main implementations:
1. Check if the `action` to be implemented is valid given the `substitutions` given. In the case that it is not, the function returns None.
2. If the function validates the substitutions, the function moves on to implement the deletions given by the action in a new `result_state`. If the deletion corresponds to a variable, then the function looks into the substitutionsto fetch the right value.
3. If the function validates the substitutions, the functions moves on to implement the additions given by the actions the new `result_state`. If the addition corresponds to a variable, then the function looks into the substitutions to fetch the right value to add. 

In [16]:
def build_state(state, action, substitutions, actions, valid=True):
    result_state = deepcopy(state)
    act = actions[action]
    to_delete = act['delete'] if 'delete' in act else []
    
    if valid:
        conditions = act['conditions'] if 'conditions' in act else []        
        for condition in conditions:  
            validation = []
            for element in condition:
                if is_variable(element):
                    validation.append(substitutions[element])
                else:
                    validation.append(element) 
            if validation not in state:
                return None
            
    for deletion in to_delete:
        to_remove = []
        for element in deletion:
            if is_variable(element):
                to_remove.append(substitutions[element])
            else:
                to_remove.append(element)
        result_state = [predicate for predicate in result_state if predicate != to_remove]
        
    to_add = act['add'] if 'add' in act else []    
    for addition in to_add:
        to_include = []
        for element in addition:
            if is_variable(element):
                to_include.append(substitutions[element])
            else:
                to_include.append(element)
        
        if to_include not in result_state:
            result_state.append(to_include)
    
    return result_state

In [17]:
# the first option is valid but yields the same state space although at a different order
assert build_state(sample_state, 'fly', sample_search_actions[0], sample_actions) == [['plane', '1973'],
                                                                                      ['plane', '2749'],
                                                                                      ['airport', 'SFO'],
                                                                                      ['airport', 'JFK'],
                                                                                      ['airport', 'ORD'],
                                                                                      ['at', '2749', 'JFK'],
                                                                                      ['at', '97', 'ORD'],
                                                                                      ['at', '1211', 'SFO'],
                                                                                      ['at', '1973', 'SFO']]
# The second to third options are not valid
for i in range(1,3):    
    assert build_state(sample_state, 'fly', sample_search_actions[i], sample_actions) == None
# the fourth option is valid and yields a valid new state
# where 1973 flies from 'SFO' to 'JFK'
assert build_state(sample_state, 'fly', sample_search_actions[3], sample_actions) == [['plane', '1973'],
                                                                                      ['plane', '2749'],
                                                                                      ['airport', 'SFO'],
                                                                                      ['airport', 'JFK'],
                                                                                      ['airport', 'ORD'],
                                                                                      ['at', '2749', 'JFK'],
                                                                                      ['at', '97', 'ORD'],
                                                                                      ['at', '1211', 'SFO'],
                                                                                      ['at', '1973', 'JFK']]

#### `build_node(current_state, previous_state, actions)`
`build_node()` creates nodes. A node connects the `current_state` with a `previous_state` and also indicates the `actions` which have been taken so far as part of the plan. The node is stored in a dictionary with the respective keys 'state','previous', and 'plan'.

In [18]:
def build_node(current_state, previous_state, actions):
    actions = [] if actions is None else actions
    return {'state':current_state, 'previous':previous_state, 'plan': actions}    

In [19]:
sample_previous_state_ = None
sample_actions_ = None
sample_node = build_node(sample_state, sample_previous_state_, sample_actions_)
assert sample_node == {'state': [['plane', '1973'],
                                 ['plane', '2749'],
                                 ['airport', 'SFO'],
                                 ['airport', 'JFK'],
                                 ['airport', 'ORD'],
                                 ['at', '1973', 'SFO'],
                                 ['at', '2749', 'JFK'],
                                 ['at', '97', 'ORD'],
                                 ['at', '1211', 'SFO']],
                       'previous': None,
                       'plan': []}

another_node = build_node(sample_state, sample_state, sample_actions_)
assert another_node == {'state': [['plane', '1973'],
                                  ['plane', '2749'],
                                  ['airport', 'SFO'],
                                  ['airport', 'JFK'],
                                  ['airport', 'ORD'],
                                  ['at', '1973', 'SFO'],
                                  ['at', '2749', 'JFK'],
                                  ['at', '97', 'ORD'],
                                  ['at', '1211', 'SFO']],
                        'previous': [['plane', '1973'],  
                                     ['plane', '2749'],  
                                     ['airport', 'SFO'], 
                                     ['airport', 'JFK'],  
                                     ['airport', 'ORD'],
                                     ['at', '1973', 'SFO'],
                                     ['at', '2749', 'JFK'],
                                     ['at', '97', 'ORD'],
                                     ['at', '1211', 'SFO']],
                        'plan': []}

#### `find_successors(node, actions)`
The function `find_successors()` does just what the name implies. It finds all succesors given a `node` and `actions`. It uses `search_actions()` and `build_state()` as previously defined to find all possible succesors. The function returns a list of all possible succesors whenever the resulting state is not None. Recall that None is passed whenever the function `build_state()` finds an invalid combination.   

In [20]:
def find_successors(node, actions):
    successors = []
    for action in actions:
        for substitution in search_actions(node['state'], action, actions):
            new_state = build_state(node['state'], action, substitution, actions)
            act = []
            new_state = build_state(node['state'], action, substitution, actions)
            for element in actions[action]['action']:
                if is_variable(element):
                    act.append(substitution[element])
                else:
                    act.append(element)
            act = node['plan'] + [act]
            successors.append(build_node(new_state, node, act))
    return [successor for successor in successors if successor['state'] is not None]

In [21]:
# from previous steps we have seen that the number of valid actions is 6
sample_successors = find_successors(sample_node, sample_actions)

In [22]:
# from previous steps we have seen that the number of valid actions is 6
sample_successors = find_successors(sample_node, sample_actions)
assert sample_successors == [{'state': [['plane', '1973'],
                                        ['plane', '2749'],
                                        ['airport', 'SFO'],
                                        ['airport', 'JFK'],
                                        ['airport', 'ORD'],
                                        ['at', '2749', 'JFK'],
                                        ['at', '97', 'ORD'],
                                        ['at', '1211', 'SFO'],
                                        ['at', '1973', 'SFO']],
                              'previous': {'state': [['plane', '1973'],
                                                     ['plane', '2749'],
                                                     ['airport', 'SFO'],
                                                     ['airport', 'JFK'],
                                                     ['airport', 'ORD'],
                                                     ['at', '1973', 'SFO'],
                                                     ['at', '2749', 'JFK'],
                                                     ['at', '97', 'ORD'],
                                                     ['at', '1211', 'SFO']],
                                           'previous': None,
                                           'plan': []},
                              'plan': [['fly', '1973', 'SFO', 'SFO']]},
                             {'state': [['plane', '1973'],
                                        ['plane', '2749'],
                                        ['airport', 'SFO'],
                                        ['airport', 'JFK'],
                                        ['airport', 'ORD'],
                                        ['at', '2749', 'JFK'],
                                        ['at', '97', 'ORD'],
                                        ['at', '1211', 'SFO'],
                                        ['at', '1973', 'JFK']],
                              'previous': {'state': [['plane', '1973'],
                                                     ['plane', '2749'],
                                                     ['airport', 'SFO'],
                                                     ['airport', 'JFK'],
                                                     ['airport', 'ORD'],
                                                     ['at', '1973', 'SFO'],
                                                     ['at', '2749', 'JFK'],
                                                     ['at', '97', 'ORD'],
                                                     ['at', '1211', 'SFO']],
                                           'previous': None,
                                           'plan': []},
                              'plan': [['fly', '1973', 'SFO', 'JFK']]},
                             {'state': [['plane', '1973'],
                                        ['plane', '2749'],
                                        ['airport', 'SFO'],
                                        ['airport', 'JFK'],
                                        ['airport', 'ORD'],
                                        ['at', '2749', 'JFK'],
                                        ['at', '97', 'ORD'],
                                        ['at', '1211', 'SFO'],
                                        ['at', '1973', 'ORD']],
                              'previous': {'state': [['plane', '1973'],
                                                     ['plane', '2749'],
                                                     ['airport', 'SFO'],
                                                     ['airport', 'JFK'],
                                                     ['airport', 'ORD'],
                                                     ['at', '1973', 'SFO'],
                                                     ['at', '2749', 'JFK'],
                                                     ['at', '97', 'ORD'],
                                                     ['at', '1211', 'SFO']],
                                           'previous': None,
                                           'plan': []},
                              'plan': [['fly', '1973', 'SFO', 'ORD']]},
                             {'state': [['plane', '1973'],
                                        ['plane', '2749'],
                                        ['airport', 'SFO'],
                                        ['airport', 'JFK'],
                                        ['airport', 'ORD'],
                                        ['at', '1973', 'SFO'],
                                        ['at', '97', 'ORD'],
                                        ['at', '1211', 'SFO'],
                                        ['at', '2749', 'SFO']],
                              'previous': {'state': [['plane', '1973'],
                                                     ['plane', '2749'],
                                                     ['airport', 'SFO'],
                                                     ['airport', 'JFK'],
                                                     ['airport', 'ORD'],
                                                     ['at', '1973', 'SFO'],
                                                     ['at', '2749', 'JFK'],
                                                     ['at', '97', 'ORD'],
                                                     ['at', '1211', 'SFO']],
                                           'previous': None,
                                           'plan': []},
                              'plan': [['fly', '2749', 'JFK', 'SFO']]},
                             {'state': [['plane', '1973'],
                                        ['plane', '2749'],
                                        ['airport', 'SFO'],
                                        ['airport', 'JFK'],
                                        ['airport', 'ORD'],
                                        ['at', '1973', 'SFO'],
                                        ['at', '97', 'ORD'],
                                        ['at', '1211', 'SFO'],
                                        ['at', '2749', 'JFK']],
                              'previous': {'state': [['plane', '1973'],
                                                     ['plane', '2749'],
                                                     ['airport', 'SFO'],
                                                     ['airport', 'JFK'],
                                                     ['airport', 'ORD'],
                                                     ['at', '1973', 'SFO'],
                                                     ['at', '2749', 'JFK'],
                                                     ['at', '97', 'ORD'],
                                                     ['at', '1211', 'SFO']],
                                           'previous': None,
                                           'plan': []},
                              'plan': [['fly', '2749', 'JFK', 'JFK']]},
                             {'state': [['plane', '1973'],
                                        ['plane', '2749'],
                                        ['airport', 'SFO'],
                                        ['airport', 'JFK'],
                                        ['airport', 'ORD'],
                                        ['at', '1973', 'SFO'],
                                        ['at', '97', 'ORD'],
                                        ['at', '1211', 'SFO'],
                                        ['at', '2749', 'ORD']],
                              'previous': {'state': [['plane', '1973'],
                                                     ['plane', '2749'],
                                                     ['airport', 'SFO'],
                                                     ['airport', 'JFK'],
                                                     ['airport', 'ORD'],
                                                     ['at', '1973', 'SFO'],
                                                     ['at', '2749', 'JFK'],
                                                     ['at', '97', 'ORD'],
                                                     ['at', '1211', 'SFO']],
                                           'previous': None,
                                           'plan': []},
                              'plan': [['fly', '2749', 'JFK', 'ORD']]}]
assert len(sample_successors) == 6

#### `goal_satisfied(goal, state)`
`goal_satisfied()` checks whether all the elements in `goal` are included in `state`. The function returns either `True` or `False` given the conditions.

In [23]:
def goal_satisfied(goal, state):
    return all(go in state for go in goal)

In [24]:
sample_goal = ['(at 1973 ORD)',
              '(at 1211 JFK)',
              '(at 2749 SFO)']
assert goal_satisfied(sample_goal, sample_successors[0]['state']) == False
trial_state = ['(at 1973 ORD)',              
              '(at 2749 SFO)',
              '(at 1211 JFK)']
# different order but same goal
assert goal_satisfied(sample_goal, trial_state) == True

#### `fetch_plan(node, debug=False)`
`fetch_plan()` corresponds to the function which traverses the nodes created in the graph to fetch the actions taken on a given succesful path. This function takes a particular `node` and a `debug` indicator. The `debug` indicator serves to print out the full node or only the action taken.

In [25]:
def fetch_plan(node, debug=False):
    plan_ = node['plan']
    
    if not debug:
        return plan_
    
    j = -1
    plan = [node['state']]
    while node['previous'] is not None:
        previous = node['previous']
        plan.append(plan_[j])
        plan.append(previous['state'])
        node = previous
        j -= 1
    plan.reverse()
    return plan

### forward_planner( start_state, goal, actions)
In order to implement the forward planner, we will use the pseudo-code provide in Module 1. We will then use a DFS approach to navigate the frontier (queue with LIFO). The difference between this implementation and simple state search is that the algorithm will implement state space while looking for possible successors, but is also another state space implemented while looking for possible actions to take. 
The implementation of the algorithm requires careful consideration of the data structures to be used. 

In [26]:
def forward_planner( start_state, goal, actions, debug=False):
    start_state = [parse(predicate) for predicate in start_state]
    goal = [parse(predicate) for predicate in goal]
    actions = parse_actions(actions)
    current_node = build_node(start_state, None, None)
    if goal_satisfied(goal, current_node['state']):
        return fetch_plan(current_node, debug)
    frontier = [current_node]
    reached = [set(["("+" ".join(s)+")" for s in start_state])]
    while len(frontier) != 0:
        current_node = frontier.pop(0)
        for child in find_successors(current_node, actions):
            state = child['state']
            if goal_satisfied(goal, current_node['state']):
                return fetch_plan(current_node, debug)
            if set(["("+" ".join(s)+")" for s in state]) not in reached:
                reached.append(set(["("+" ".join(s)+")" for s in state]))
                frontier.append(child)
    return None

In [27]:
# This corresponds to problem 3 in the self-check
sample_actions = {'fly':{'action':"(fly ?plane ?from ?to)",
                       'conditions':['(plane ?plane)','(airport ?to)','(airport ?from)',
                                     '(at ?plane ?from)','(fueled ?plane)'],
                       'add':['(at ?plane ?to)'],
                       'delete':['(at ?plane ?from)']},
                 'fuel':{'action':"(fuel ?plane)",
                        'conditions':['(plane ?plane)','(unfueled ?plane)'],
                        'add':['(fueled ?plane)'],
                        'delete': ['(unfueled ?plane)']}}
sample_state = ['(plane 1973)','(plane 2749)','(plane 97)','(plane 1211)',
                '(airport SFO)','(airport JFK)','(airport ORD)',
                '(at 1973 SFO)','(at 2749 JFK)','(at 97 ORD)','(at 1211 SFO)',
                '(fueled 1973)','(unfueled 2749)','(unfueled 97)', '(fueled 1211)']

sample_goal = ['(at 1973 ORD)',
              '(at 1211 JFK)',
              '(at 2749 ORD)']
plan = forward_planner( sample_state, sample_goal, sample_actions, False)

for el in plan:
    print(el)

['fly', '1973', 'SFO', 'ORD']
['fly', '1211', 'SFO', 'JFK']
['fuel', '2749']
['fly', '2749', 'JFK', 'ORD']


In [28]:
plan = forward_planner(start_state, goal, actions, False)
for el in plan:
    print(el)

['drive', 'Me', 'Home', 'Store']
['buy', 'Me', 'Store', 'Drill']
['drive', 'Me', 'Store', 'Home']


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.